In [1]:
import sys
sys.path.append("/home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [1]:
import numpy as np
import pandas as pd

In [2]:
Fasta = pd.read_csv("mm10_Fasta_Non_Donor_Acceptor.csv")

In [3]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq
0,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,1933,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...
1,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2301,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...
2,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,3329,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...
3,Arhgap21,chr2,-,20847918,20967721,20848711,20967410,25,"20847918,20853037,20854934,20855353,20855580,2...","20850385,20853069,20855040,20855395,20855706,2...",...,2467,2652,32,114652,GGAAGGCTCTCCGAAAGCTGATTTTTCTCTGGCCTCCGAGTAACAC...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAG,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,GGCTCCTGGGGATCCGGGAAGGATCAGTATAGCAGGGAACTGCTTG...,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAGCAAAGTTGTTCTAT...
4,Ccnj,chr19,+,40831278,40848572,40832019,40846098,6,"40831278,40831978,40836916,40844659,40845044,4...","40831442,40832088,40837127,40844959,40845204,4...",...,2874,494,160,13766,GTACGGCGAGCCGCCTGCCGGTCCTTTAACAATGGGCGGCGCGAGC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,TGCTCACGATAACGACGTGAAAGAAGCAAACAAGCAGAGGGGACAG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,Wnt4,chr4,+,137277488,137299726,137277679,137296779,5,"137277488,137289113,137295440,137295659,137296...","137277756,137289349,137295572,137295802,137299...",...,3415,509,143,18171,GGCTCTGGGGCGGCGCTGACAGTCGGGTCCGCCGGGCAGCGGGCGC...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCCATCTTGACACACATGCGGGTGGAGTGCAAGTGTCACGGGGTGT...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...
2366,Inpp4a,chr1,+,37299890,37410736,37357918,37408259,26,"37299890,37357342,37357814,37358507,37364109,3...","37300118,37357404,37358024,37358552,37364228,3...",...,2610,9206,155,98875,GCGGGGTTGTGCGCTGTGTTCTGAGAGAAGGCGGTGGCAGGATTCG...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,TGAGGGTTGTCGGCGAGAAAACACAATGAAGAATGTTGGAAGTCGC...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...
2367,Trps1,chr15,-,50654751,50889691,50660674,50846940,6,"50654751,50664686,50822104,50830828,50846013,5...","50661736,50664809,50822708,50831958,50846942,5...",...,6985,2950,123,224882,TGCAAACAACAGAAGACGGTTCATGGCTCTGGCCGCCGCGCCACCA...,AGGCGAAGAGGATCTGGTGTTTTTTGTGCCAATTGCCTGACCACAA...,GTAAGAACTTATCCTAGCCCTTTAGGGGAA

### Select positive strand

In [4]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [5]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq
0,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,1933,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...
1,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2301,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...
2,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,3329,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...
4,Ccnj,chr19,+,40831278,40848572,40832019,40846098,6,"40831278,40831978,40836916,40844659,40845044,4...","40831442,40832088,40837127,40844959,40845204,4...",...,2874,494,160,13766,GTACGGCGAGCCGCCTGCCGGTCCTTTAACAATGGGCGGCGCGAGC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,TGCTCACGATAACGACGTGAAAGAAGCAAACAAGCAGAGGGGACAG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...
5,Ky,chr9,+,102505749,102546239,102506142,102542781,11,"102505749,102509110,102512972,102521288,102523...","102506278,102509173,102513035,102521363,102523...",...,4354,2697,191,33248,TCTTAGGGGATAGATGGTGGCTATGAGGCAGCACTCAGCCCAGCCA...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,TCAATGGGAAAGCCACAATCACCATCGAGAGCCGGGCCCCAACACT...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,Zfp710,chr7,+,80024813,80092751,80081076,80090504,5,"80024813,80081048,80085950,80086433,80090334,","80024986,80082540,80086142,80086608,80092751,",...,2417,3726,175,61620,ggaggaaggagggaaggggaaggaagagcgcagggaggggagagga...,GTGTGCGGGAAGTCCTTCAACCGCATGTACAACCTGCTGGGTCATA...,GTGGGTGGGCTGTGCGCAGTGGACAGAGCAGGATTGACAGCAGCAT...,ACCCCATGATGGAGCTGGCAGGCCCTGACCCCTCTGAACTTGACAA...,GTGGGTGGGCTGTGCGCAGTGGACAGAGCAGGATTGACAGCAGCAT...,GTGTGCGGGAAGTCCTTCAACCGCATGTACAACCTGCTGGGTCATA...
2364,Ptchd4,chr17,+,42315946,42507741,42316304,42503915,4,"42315946,42316654,42377159,42502281,","42316309,42317240,42377640,42507741,",...,5460,124641,481,61213,CTGCTCCAATGGGCCAGCAGCGCCTTCACCCTTTACGTCTCCCATT...,GATGGGAGGAACAGTTTTATTGGACACCAGCTGGGCGGGGTAGTGG...,GTAACTATCCATCCTTGTGGTAATCGAATTCTTGCCTGTTGGGGGC...,GTCATGGAACTAAAGGAGTATTTGAGCTTCTGTCTGGATGGCGGAG...,GTAACTATCCATCCTTGTGGTAATCGAATTCTTGCCTGTTGGGGGC...,GATGGGAGGAACAGTTTTATTGGACACCAGCTGGGCGGGGTAGTGG...
2365,Wnt4,chr4,+,137277488,137299726,137277679,137296779,5,"137277488,137289113,137295440,137295659,137296...","137277756,137289349,137295572,137295802,137299...",...,3415,509,143,18171,GGCTCTGGGGCGGCGCTGACAGTCGGGTCCGCCGGGCAGCGGGCGC...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG..

In [26]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [ ]:
PreSequence = Fasta_Pos["PreSequence"].tolist()
Sequence = Fasta_Pos["Seq"].tolist()

In [28]:
df = pd.DataFrame(range(-1000,1001),columns=["index"])

In [34]:
for i in range(len(Fasta_Pos)):
    
    # Define exon location   
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    Length = Length[:-3]
    Length.append(Fasta_Pos.loc[i,"SecondToLastLength"] + 100 + Fasta_Pos.loc[i,"LastExonLength"])
    
    # transcript
    Pre = PreSequence[i]
    Seq = Sequence[i]
    
    # Define seq
    SecondToLastExon = Seq[0:(Fasta_Pos.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Pos.loc[i,"SecondToLastLength"]):(Fasta_Pos.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Pos.loc[i,"LastExonLength"]):]   
    
    # Intron sequence scramble
    Intron = list(Intron)
    random.shuffle(Intron)
    Intron = "".join(Intron)    

    input_sequence = Pre + SecondToLastExon + Intron + LastExon  
    
    ## Predict transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    
    # Define prob in exons
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)    
    
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)    
 
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    Last = Last[0:(Fasta_Pos.loc[i,"SecondToLastLength"])] + Last[-(Fasta_Pos.loc[i,"LastExonLength"]):]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)

### Select negative strand

In [16]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [17]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [ ]:
PreSequence = Fasta_Neg["PreSequence"].tolist()
Sequence = Fasta_Neg["Seq"].tolist()

In [ ]:
for i in range(len(Fasta_Neg)):
    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    Length = Length[:-3]
    Length.append(Fasta_Neg.loc[i,"SecondToLastLength"] + 100 + Fasta_Neg.loc[i,"LastExonLength"])
    
    # transcript
    Pre = PreSequence[i]
    Seq = Sequence[i]
    
    # Define seq
    SecondToLastExon = Seq[0:(Fasta_Neg.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Neg.loc[i,"SecondToLastLength"]):(Fasta_Neg.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Neg.loc[i,"LastExonLength"]):]   
    
    # Intron sequence scramble
    Intron = list(Intron)
    random.shuffle(Intron)
    Intron = "".join(Intron)    

    input_sequence = Pre + SecondToLastExon + Intron + LastExon
    
    ## Predict transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define prob in exons
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    Last = Last[0:(Fasta_Neg.loc[i,"SecondToLastLength"])] + Last[-(Fasta_Neg.loc[i,"LastExonLength"]):]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)            

In [ ]:
df

In [ ]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Sum"] = df.sum()
df = df.T

In [ ]:
Value = df[["Sum"]]

In [ ]:
Value.head(5)

In [ ]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [ ]:
Number = df[["Number"]]

In [ ]:
Number.head(5)

In [ ]:
Data = pd.concat([Value, Number], axis=1)

In [ ]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [ ]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [ ]:
Result.to_csv("iM6A_Mouse_NewLastExonStart_TruncationWithSpliceMutation_ShuffleIntron_WithoutIntron.csv", index=0)